In [ ]:
# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import os, pathlib

BASE = '..'
dirs = [
    f'{BASE}/data/raw',
    f'{BASE}/data/processed',
    f'{BASE}/reports/figures',
    f'{BASE}/src',
]
for d in dirs:
    pathlib.Path(d).mkdir(parents=True, exist_ok=True)
dirs

In [ ]:
# setting databases, importing stuff
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
import numpy as np
from scipy.stats import chi2_contingency
import re
%matplotlib inline

BASE = Path('..')
RAW_DIR = BASE / 'data/raw'
OUT_DIR = BASE / 'data/processed'
FIG_DIR = BASE / 'reports/figures'
OUT_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

assets = pd.read_csv(OUT_DIR / "asset_information_final.csv") # read in the processed asset information instead, with all the different descriptions, because they have been generated
customers = pd.read_csv(RAW_DIR / "customer_information.csv")
markets = pd.read_csv(RAW_DIR / "markets.csv")
transactions = pd.read_csv(RAW_DIR / "transactions.csv")
closeprices = pd.read_csv(RAW_DIR / "close_prices.csv")
limits = pd.read_csv(RAW_DIR / "limit_prices.csv")
# questionnaires = pd.read_csv(RAW_DIR / "questionnaires.csv")

CONFIG = {
    "id_col": "customer_id",
    "predicted_flag_col": "is_predicted",
    "pred_label_col": "riskLevel_pred",
    "true_label_col": "riskLevel_true",
    "investment_capacity_col": "investmentCapacity",
    "demographics": {
        "age": "age",
        "gender": "gender",
        "region": "region",
        "income": "income",
    },
    "value_metric_col": "annual_value",
    "pred_proba_cols": {}
}


## Data Cleaning

### Check for duplicates

Helper functions

In [ ]:
# Function to print duplicate info
def check_duplicates(df, name, subset_cols=None):
    total_dupes = df.duplicated().sum()
    if subset_cols:
        key_dupes = df.duplicated(subset=subset_cols).sum()
        print(f"{name}: {total_dupes} total duplicate rows | {key_dupes} duplicate(s) on {subset_cols}")
    else:
        print(f"{name}: {total_dupes} total duplicate rows")

def resolve_asset_conflicts(df):
    """
    Resolve conflicts between assets.
    Groups by (ISIN, timestamp), keeping the first row,
    except prioritizing 'MTF' in assetCategory when conflicts exist.
    """
    # Sort so first occurrence is consistent
    df_sorted = df.sort_values(['ISIN', 'timestamp']).reset_index(drop=True)

    # Handle assetCategory priority: ensure 'MTF' appears first within each group if present
    df_sorted['assetCategory'] = df_sorted.groupby(['ISIN','timestamp'])['assetCategory']\
                                          .transform(lambda x: 'MTF' if 'MTF' in x.values else x.iloc[0])

    # Drop duplicates keeping first row (after adjusting assetCategory)
    clean = df_sorted.drop_duplicates(['ISIN','timestamp'], keep='first')

    return clean


In [ ]:
# Check duplicates in each dataset
check_duplicates(customers, "Customer Info", subset_cols=["customerID"])
check_duplicates(assets, "Asset Info", subset_cols=["ISIN"])
check_duplicates(markets, "Markets", subset_cols=["marketID"])
check_duplicates(closeprices, "Close Prices", subset_cols=["ISIN", "timestamp"])
check_duplicates(limits, "Limit Prices", subset_cols=["ISIN"])
check_duplicates(transactions, "Transactions", subset_cols=["transactionID","customerID"])
print("-" * 30)
assets.info()

In [ ]:
# Handling Duplicates -> keep only the most updated customer and asset row
customers = customers.sort_values(["customerID", "timestamp"]).drop_duplicates("customerID", keep="last")
assets = assets.sort_values(["ISIN","timestamp"]).drop_duplicates("ISIN", keep="last")

# check duplicate rows for customer and asset
check_duplicates(customers, "Customer Info", subset_cols=["customerID"])
check_duplicates(assets, "Asset Info", subset_cols=["ISIN"])

### Check null values

In [ ]:
# find the number for null values in each dataframe
datasets = {
    "Customer Info": customers,
    "Asset Info": assets,
    "Markets": markets,
    "Close Prices": closeprices,
    "Limit Prices": limits,
    "Transactions": transactions
}

for name, df in datasets.items():
    print(f"\n=== {name} ===")
    null_counts = df.isnull().sum()
    null_percent = (df.isnull().mean() * 100).round(2)
    summary = pd.DataFrame({"Null Count": null_counts, "Null %": null_percent})
    display(summary[summary["Null Count"] > 0])

asset information and markets have null values

#### Handle null values for assets

In [ ]:
# Handling NaN for Asset Names
assets.loc[assets['ISIN'] == 'DE000A2TEDB8', 'assetName'] = 'thyssenkrupp AG'
assets.loc[assets['ISIN'] == 'LU0671501806', 'assetName'] = 'Schroder ISF Global High Yield A Dis EUR H QV'
assets.loc[assets['ISIN'] == 'US00214Q1040', 'assetName'] = 'The ARK INNOVATION ETF'
assets.loc[assets['ISIN'] == 'US0032601066', 'assetName'] = 'abrdn Physical Platinum Shares ETF'
assets.loc[assets['ISIN'] == 'US26924G7714', 'assetName'] = 'ETFMG TRAVEL TECH ETF'
assets.loc[assets['ISIN'] == 'US3814305450', 'assetName'] = 'The Goldman Sachs Hedge Industry VIP ETF'
assets.loc[assets['ISIN'] == 'US46090E1038', 'assetName'] = 'Invesco QQQ'
assets.loc[assets['ISIN'] == 'US4642863504', 'assetName'] = 'iShares MSCI Agriculture Producers ETF'
assets.loc[assets['ISIN'] == 'US4642864007', 'assetName'] = 'iShares MSCI Brazil ETF'
assets.loc[assets['ISIN'] == 'US4642871762', 'assetName'] = 'iShares TIPS Bond ETF'
assets.loc[assets['ISIN'] == 'US4642876555', 'assetName'] = 'iShares Russell 2000 ETF'
assets.loc[assets['ISIN'] == 'US4642876894', 'assetName'] = 'iShares Russell 3000 ETF'
assets.loc[assets['ISIN'] == 'US4642882579', 'assetName'] = 'iShares MSCI ACWI ETF'
assets.loc[assets['ISIN'] == 'US46429B2676', 'assetName'] = 'iShares U.S. Treasury Bond ETF'
assets.loc[assets['ISIN'] == 'US5007674055', 'assetName'] = 'KraneShares Bosera MSCI China A 50 Connect Index ETF'
assets.loc[assets['ISIN'] == 'US72201R7750', 'assetName'] = 'PIMCO Active Bond Exchange-Traded Fund'
assets.loc[assets['ISIN'] == 'US74347X8496', 'assetName'] = 'ProShares Short 20+ Year Treasury'
assets.loc[assets['ISIN'] == 'US78462F1030', 'assetName'] = 'SPDR® S&P 500® ETF Trust'
assets.loc[assets['ISIN'] == 'US78468R6633', 'assetName'] = 'Bloomberg 1-3 Month US Treasury Bill Index'
assets.loc[assets['ISIN'] == 'US81369Y5069', 'assetName'] = 'Energy Select Sector SPDR Fund'
assets.loc[assets['ISIN'] == 'US92189F1066', 'assetName'] = 'VanEck Gold Miners ETF'
assets.loc[assets['ISIN'] == 'US9219378356', 'assetName'] = 'Vanguard Total Bond Market ETF'
assets.loc[assets['ISIN'] == 'US9229085538', 'assetName'] = 'Vanguard REIT ETF'
assets.loc[assets['ISIN'] == 'US97717W8516', 'assetName'] = 'WisdomTree Japan Hedged Equity Fund'
assets.loc[assets['ISIN'] == 'GRF000394004', 'assetShortName'] = 'DELEI5Y'

# Handling Null in Asset SubCategory
assets['assetSubCategory'] = assets.apply(
    lambda row: 'Stock' if (row['assetCategory'] == 'Stock' and pd.isna(row['assetSubCategory']))
    else row['assetSubCategory'],
    axis=1
)

# Handling NaN in Sector and Industry for MTF and Bonds
mask = assets['assetCategory'].isin(['MTF', 'Bond'])

assets.loc[mask & assets['sector'].isna(), 'sector'] = 'NIL'
assets.loc[mask & assets['industry'].isna(), 'industry'] = 'NIL'

# Handling NaN in Sector and Industry for Stocks
mask_remaining = (
    (assets['assetCategory'] == 'Stock') &
    (assets[['sector', 'industry']].isna().any(axis=1))
)

assets.loc[mask_remaining, 'sector'] = 'Unknown'
assets.loc[mask_remaining, 'industry'] = 'Unknown' # Can be mapped for better accuracy later


### Explore correlation between predicted risk level and investment capacity

In [ ]:
# find predicted risk level and predicted capacity
customers["is_predicted_risk"] = customers["riskLevel"].fillna("").str.contains("Predicted", case=False)
customers["is_predicted_capacity"] = customers["investmentCapacity"].fillna("").str.contains("Predicted", case=False)

# display values
display(customers["is_predicted_risk"].value_counts(normalize=True))
print("-" * 50)
display(customers["is_predicted_capacity"].value_counts(normalize=True))

Large proportion of customers have either predicted risk level or predicted investment capacity

In [ ]:
print("Predicted Customers (risk level):\n")
print(customers.loc[customers["is_predicted_risk"], "riskLevel"].value_counts(), "\n")
print("-" * 50)
print("Non Predicted Customers (risk level):\n")
print(customers.loc[~customers["is_predicted_risk"], "riskLevel"].value_counts(), "\n")

In [ ]:
print("Predicted Customers (Investment Capacity): \n")
print(customers.loc[customers["is_predicted_capacity"], "investmentCapacity"].value_counts(), "\n")
print("-" * 50)
print("Non Predicted Customers (Investment Capacity): \n")
print(customers.loc[~customers["is_predicted_capacity"], "investmentCapacity"].value_counts(), "\n")

### Check association between predicted risk and predicted investment capacity

In [ ]:
# heatmap of risk vs capacity (predicted)
ct = pd.crosstab(
    customers.loc[customers["is_predicted_risk"], "riskLevel"],
    customers.loc[customers["is_predicted_capacity"], "investmentCapacity"],
    normalize='index'
)
# plot heatmap
sns.heatmap(ct, annot=True, cmap="Blues")
plt.title("Predicted Risk vs Predicted Capacity")
plt.show()

There could be some association between predicted risk levels and predicted investment capacity

### Check association between non-predicted risk and non-predicted investment capacity

In [ ]:
# heatmap of risk vs capacity (predicted)
ct = pd.crosstab(
    customers.loc[~customers["is_predicted_risk"], "riskLevel"],
    customers.loc[~customers["is_predicted_capacity"], "investmentCapacity"],
    normalize='index'
)
# plot heatmap
sns.heatmap(ct, annot=True, cmap="Blues")
plt.title("Actual risk level vs actual investment capacity")
plt.show()

We can observe that if investment capacity is not available, risk is also not available.

Most customers among all risk levels have investment capacity CAP_LT30K

### Predicted vs non-predicted counts

In [ ]:
customers_copy = customers.copy().reset_index(drop=True)
customers_copy['is_predicted_risk'].value_counts().plot(kind='bar')
plt.title("Customer risk level")
plt.tight_layout()
plt.show()

In [ ]:
customers_copy['is_predicted_capacity'].value_counts().plot(kind='bar')
plt.title("Customer investment capacity")
plt.tight_layout()
plt.show()

In [ ]:
customers_copy['riskLevel'].value_counts().plot(kind='bar')
plt.title("Customer risk levels")
plt.tight_layout()
plt.show()

### Explore each customer's buy-sell ratio

In [ ]:
# Group by customer and transaction type
counts = (
    transactions.groupby(["customerID", "transactionType"])
      .size()
      .unstack(fill_value=0)  # ensures both BUY and SELL appear
      .reset_index(drop=True)
)

print(counts)

In [ ]:
# Create Buy/Sell ratio
counts["total"] = counts["Buy"] + counts["Sell"]
counts["buy_ratio"] = counts["Buy"] / counts["total"]
counts["sell_ratio"] = counts["Sell"] / counts["total"]
# counts.to_csv("customer_buy_sell_ratios.csv", index=False)

In [ ]:
# Distribution of buy/sell ratios
plt.figure(figsize=(8, 5))
plt.hist(counts["buy_ratio"], bins=20, color="steelblue", edgecolor="black")
plt.title("Distribution of Buy Ratios per Customer")
plt.xlabel("Buy Ratio (0 = All Sells, 1 = All Buys)")
plt.ylabel("Number of Customers")
plt.show()

In [ ]:
# Assuming counts["buy_ratio"] already exists, convert the buy ratios into 10 bins

# Define bins (0 to 1 in steps of 0.1)
bins = [i/10 for i in range(11)]  # [0.0, 0.1, ..., 1.0]
labels = [f"{bins[i]:.1f}-{bins[i+1]:.1f}" for i in range(len(bins)-1)]

# Cut buy_ratios into bins
counts["buy_ratio_bin"] = pd.cut(counts["buy_ratio"], bins=bins, labels=labels, include_lowest=True)

# Calculate distribution
summary = (
    counts["buy_ratio_bin"]
    .value_counts(normalize=True, sort=False) * 100
).reset_index()

summary.columns = ["Buy Ratio Range", "% of Customers"]

print(summary)


Most customers have a ratio of around 0.4 to 0.5

### Explore the transaction counts over time

In [ ]:
transactions["timestamp"] = pd.to_datetime(transactions["timestamp"])
transactions["year_month"] = transactions["timestamp"].dt.to_period("M")

monthly = (
    transactions.groupby(["year_month", "transactionType"])
      .size()
      .unstack(fill_value=0)
)

monthly.plot(kind="bar", stacked=True, figsize=(12,6), colormap="coolwarm")
plt.title("Monthly Buy/Sell Transaction Counts")
plt.xlabel("Year-Month")
plt.ylabel("Number of Transactions")
plt.show()

Most customers purchase/sell assets in 2018

In [ ]:
customer_activity = (
    transactions.groupby("customerID")["transactionID"]
      .count()
      .reset_index(name="transaction_count")
      .sort_values(by="transaction_count", ascending=False)
)

print(customer_activity.head(10))  # Top 10 most active customers

In [ ]:
asset_diversity = (
    transactions.groupby("customerID")["ISIN"]
      .nunique()
      .reset_index(name="unique_assets")
)

print(asset_diversity.describe())  # summary stats


Mean number of assets each customer owns = 3

### Explore holding periods for each customer

In [ ]:
# Only keep buy and sell actions
buys = transactions[transactions["transactionType"] == "Buy"]
sells = transactions[transactions["transactionType"] == "Sell"]

# Merge on customer + asset (ISIN = asset identifier)
merged = pd.merge(
    buys, sells,
    on=["customerID", "ISIN"],
    suffixes=("_buy", "_sell")
)

# Make sure timestamps are datetime
merged["timestamp_buy"] = pd.to_datetime(merged["timestamp_buy"])
merged["timestamp_sell"] = pd.to_datetime(merged["timestamp_sell"])

# Compute holding period (only valid if sell date is after buy date)
merged = merged[merged["timestamp_sell"] > merged["timestamp_buy"]]
merged["holding_days"] = (merged["timestamp_sell"] - merged["timestamp_buy"]).dt.days

print(merged[["customerID", "ISIN", "holding_days"]].head())
print("-" * 50)
print(merged["holding_days"].describe())

In [ ]:
# Overall stats across ALL customers & assets
mean_holding = merged["holding_days"].mean()
median_holding = merged["holding_days"].median()

print(f"Average holding period: {mean_holding:.1f} days")
print(f"Median holding period: {median_holding:.1f} days")
print("-" * 50)
print(merged["holding_days"].describe(percentiles=[0.25, 0.5, 0.75]))

Holding period seems to be right-skewed

In [ ]:
plt.figure(figsize=(15,10))
sns.histplot(merged["holding_days"], kde=True, bins=100)
plt.title("Distribution of Holding Periods")
plt.xlabel("Holding Period (days)")
plt.ylabel("Frequency")
plt.show()

Most holding periods < 250 days

### Transaction total values by asset

In [ ]:
volume_by_asset = (
    transactions.groupby("ISIN")["totalValue"]
               .sum()
               .reset_index(name="total_volume")
               .sort_values(by="total_volume", ascending=False)
)

In [ ]:
tx_assets = transactions.merge(assets, on="ISIN", how="left")
volume_by_category = (
    tx_assets.groupby("assetCategory")["totalValue"]
             .sum()
             .reset_index(name="total_volume")
             .sort_values(by="total_volume", ascending=False)
)

print(volume_by_category)

Most people in the dataset are trading bonds

### Merge all data into 1 dataframe

In [ ]:
display(customers.head(3))

In [ ]:
display(transactions.head(3))

In [ ]:
customers = customers.rename(columns={'timestamp': 'account_creation_date'}).drop(columns=['is_predicted_risk', 'is_predicted_capacity'])
transactions = transactions.rename(columns={'timestamp': 'transaction_date'}).drop(columns='year_month')

In [ ]:
merged = pd.merge(customers, transactions, on='customerID', how='inner', suffixes=('_customer', '_transaction'))
display(merged.head(3))

In [ ]:
display(assets.head(3))

In [ ]:
merged = pd.merge(merged, assets.reset_index(drop=True).drop(columns=['timestamp', 'tavily_search_results', 'content', 'errors', 'marketID']), on=['ISIN'], how='inner')
display(merged.head(3))

In [ ]:
merged.shape

In [ ]:
merged.columns

In [ ]:
display(markets.head(3))

In [ ]:
merged = pd.merge(merged.rename(columns={'description': 'asset_description'}), markets.rename(columns={'description': 'market_description'}), on='marketID', how='inner')
display(merged.head(3))

In [ ]:
print(merged.columns)

In [ ]:
merged = merged.drop(columns=['assetShortName', 'ticker', 'tradingDays', 'tradingDays', 'lastQuestionnaireDate'])

In [ ]:
merged.to_csv(OUT_DIR / 'cleaned_data_final.csv', index=False)